# Weekly Report Generator

In [112]:
#import all relevant packages
import pandas as pd
import pytz
import numpy as np
import datetime as dt
import openpyxl
import xlsxwriter
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

In [113]:
# ask user to input a date for the calculations to run from:

year = int(input('Enter a year '))
month = int(input('Enter a month '))
day = int(input('Enter a day '))
date1 = dt.date(year, month, day)
datename = str(year)+str(month)+str(day)
print(datename)

Enter a year 2021
Enter a month 05
Enter a day 04
202154


### Read in Data

In [276]:
#read in the xlsx file for the most recent week's data
df = pd.read_csv("raw_data.csv")

In [277]:

# shape before non-null closed date rows are removed
rowcount_before = df.shape[0]
print('Number of Entries BEFORE Removing Rows with a Non-Null Closed Date ' + str(df.shape[0]))

#removed all with closed dates
df = df[df['Closed Date'].isnull()]
rowcount_after = df.shape[0]
print('Number of Entries AFTER Removing Rows with a Non-Null Closed Date ' + str(df.shape[0]))

#Total Number of rows removed
print('Number of Rows Removed: ' + str(rowcount_before - rowcount_after))

#read in the csv file will all work holidays and convert to datetime
dfhol = pd.read_csv('tbl_holidays.csv')

dfhol['HoliDate'] =pd.to_datetime(dfhol['HoliDate'])


#add on the dates before any calculations are made
df['today'] = pd.to_datetime(date1)

# Use index sort to ensure that the data is organized (redundant Step)
df.sort_index(inplace = True)
print(dfhol[200:250])

Number of Entries BEFORE Removing Rows with a Non-Null Closed Date 538
Number of Entries AFTER Removing Rows with a Non-Null Closed Date 538
Number of Rows Removed: 0
      HoliDate
200 2019-07-05
201 2019-09-02
202 2019-11-11
203 2019-11-28
204 2019-11-29
205 2019-12-24
206 2019-12-25
207 2019-12-26
208 2019-12-27
209 2019-12-31
210 2020-01-01
211 2020-01-20
212 2020-02-17
213 2020-05-25
214 2020-07-04
215 2020-09-07
216 2020-11-11
217 2020-11-26
218 2020-11-27
219 2020-12-24
220 2020-12-25
221 2020-12-26
222 2021-01-01
223 2021-01-18
224 2021-02-15
225 2021-02-16
226 2021-02-17
227 2021-02-18
228 2021-02-19
229 2021-04-02
230 2021-05-31
231 2021-07-04
232 2021-09-06
233 2021-11-11
234 2021-11-25
235 2021-11-26
236 2021-12-24
237 2021-12-25
238 2021-12-26
239 2022-01-01
240 2022-01-17
241 2022-02-21
242 2022-05-30
243 2022-07-04
244 2022-09-05
245 2022-11-11
246 2022-11-24
247 2022-11-25
248 2022-12-24
249 2022-12-25


In [278]:
#Change status to category datatype
df['Status'] = df['Status'].astype('category')
#SNAPPER Issue converted to Category (is actually a Boolean using Y and N)
df['SNAPPER'] = df['SNAPPER Issue'].astype('category')

#Split the case type descriptor from the case type category number for easier parsing later
df[['case_type_num', 'case_type_desc']] = df['Case Type'].str.split(' - ', expand = True)
print('Case Types Numbers Generated')

#Convert to category datatypes
df['case_type_num'] = pd.to_numeric(df['case_type_num'], errors='coerce', downcast = 'integer').astype('category')
df['case_type_desc'] = df['case_type_desc'].astype('category')

Case Types Numbers Generated


In [279]:
df['Intake Date'] = pd.to_datetime(df['Intake Date'])

    
df['Date Response Submitted'] = pd.to_datetime(df['Date Response Submitted'])


df['90th Calendar Date'] = df['Intake Date'] + pd.DateOffset(days=89)
df['Report Due to Program Coordinator'] = df['Intake Date'] + pd.DateOffset(days=89-14)
#timedeltas
df['Calendar Days'] = df['today'] + pd.DateOffset(days=1) - df['Intake Date']
df['Days Since Response Submitted'] = df['today'] + pd.DateOffset(days=1) - df['Date Response Submitted']
# Count the number of days inbetween intake and the current date with holidays excluded

In [280]:
#create function to create mask and cover unwanted NATs in the entries, done because the xls file has a header of 6 and a footer of 3 with non time values
def business_days(start, end):
    mask = pd.notnull(start) & pd.notnull(end) #mask for where both the start and end dates are not NULL
    start = df['Intake Date'].values.astype('datetime64[D]')[mask]
    end = df['today'].values.astype('datetime64[D]')[mask]
    result = np.empty(len(mask), dtype=float)
    result[mask] = np.busday_count(start, end, holidays = dfhol['HoliDate'].values.astype('datetime64[D]')) #calculates the number of business days based on m-f with state holidays listed in dfhol['Holidate'] removed as well
    result[~mask] = np.nan #replaces all other entries with NULL
    return result

df['Work Days Between'] = business_days(df['Intake Date'], df['today'])
# Add Number of Business Days to Given Date
bday_txgov = pd.offsets.CustomBusinessDay(holidays=dfhol['HoliDate'])
df['90 Business Days'] = df['Intake Date']+ 90 * bday_txgov
df['90 Business Days minus 1'] = df['Intake Date']+ 90 * bday_txgov + pd.DateOffset(days=1)
df['nb_months'] = ((df['today'] - df['Intake Date'])/np.timedelta64(1, 'M'))
df['nb_months'] = df['nb_months'].astype(int)

print('Time Deltas Created')

Time Deltas Created


C:\Users\JFaulk01\Miniconda3\lib\site-packages\pandas\core\arrays\datetimes.py:702: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex
  warnings.warn(


##  Print Out Any Entries with Issues

In [281]:
# Negative Work Days Between
negative_work_days_with_tn = df['Tracking Number'][(df['Work Days Between'] <0)]
print(negative_work_days_with_tn)

Series([], Name: Tracking Number, dtype: object)


## Case Types Defined

In [282]:
#Define the case types allowed in each report
case_types_pending_fns_report = [14,  15,  52,  11,  12,  13,  22,  23,  24,  25,  31,  32,  33,  34,  41,  51,  71]
case_types_pending_nonfns_report = [22, 26, 31, 34, 36]
case_types_all_past_due_rar_pending_report = [41]
case_types_staff_pending_admin_complaints = [51]
case_types_all_staff_pending_case_counts = [26, 36, 16, 14, 15, 52, 11, 12, 13, 22, 23, 24, 25, 31, 32, 33, 34, 41, 51, 71]
case_types_all_staff_pending_client_cases = [26, 36, 16, 22, 23, 24, 25, 31, 32, 33, 34]
case_types_all_staff_pending_edcs_for_ops_program_mgr = [11, 14, 15, 52]
case_types_all_staff_pending_employment_cases = [11, 14, 15, 52]
case_types_all_staff_weekly_pending_cases = [26, 36, 16, 12, 23, 32, 14, 15, 52, 11, 13, 22, 24, 25, 31, 33, 34, 41, 51, 71]
case_types_fns_pend = [14, 15, 52, 11, 12, 13, 22, 23, 24, 25, 31, 32, 33, 34, 41, 51, 71]
case_types_open_charge = [12]
case_types_pend_report_ovr_60 = [26, 36, 16, 12, 23, 32, 14, 15, 52, 11, 13, 22, 24, 25, 31, 33, 34, 41, 51, 71]
case_types_pending_report_over_60_days_minus_12_23_26_32_36 = [31, 16, 14, 15, 52, 11, 13, 22, 24, 25, 33, 34, 41, 51, 71]
case_types_pending_report_over_60_days_minus_26s_31s = [36, 16, 12, 23, 32, 14, 15, 52, 11, 13, 22, 24, 25, 33, 34, 41, 51, 71]
case_types_rar = [41]
case_types_received_during_month = [26, 36, 16, 14, 15, 52, 11, 12, 13, 22, 23, 24, 31, 32, 33, 41, 51, 71]
case_types_z_all_fns_pending_tracker = [22, 23, 26, 31, 32, 34, 36]
case_types_z_all_staff_pending_client_cases = [22, 23, 26, 31, 32, 34, 36, 22, 23, 26, 31, 32, 34, 36]


#Define all the columns used in each report
col_names_pending_fns_report = ['case_type_num','Staff Assigned To', 'Tracking Number', 'Case Type',  'Intake Date', 'Status', 'SNAPPER Issue', 'Date Sent to FNS', 'Calendar Days', 'Report Due to Program Coordinator', '90th Calendar Date']

col_names_pending_nonfns_report = ['case_type_num','Staff Assigned To', 'Tracking Number', 'Case Type',  'Intake Date', 'Status', 'SNAPPER Issue', 'Date Sent to FNS', 'External Agency', 'Date Response Submitted']

col_names_all_past_due_rar_pending_report = ['case_type_num','Staff Assigned To', 'Tracking Number', 'Case Type',  'Intake Date', 'Status', 'Work Days Between', 'Complainant Agency']

col_names_staff_pending_admin_complaints = ['case_type_num','Staff Assigned To', 'Tracking Number', 'Case Type',  'Intake Date','Status','Work Days Between', 'External Agency', 'Date Response Submitted', 'Complainant Agency', 'Respondent Agency']

col_names_all_staff_pending_case_counts = ['case_type_num', 'Staff Assigned To', 'Case Type', 'Tracking Number', 'Closed Date']

col_names_all_staff_pending_client_cases = ['case_type_num', 'Staff Assigned To', 'Tracking Number', 'Case Type',  'Intake Date', 'Status', 'External Agency', 'Date Response Submitted', 'Work Days Between']

col_names_all_staff_pending_edcs_for_ops_program_mgr = ['case_type_num', 'Case Type', 'Tracking Number', 'Intake Date',  'Region', 'Staff Assigned To', 'Status']

col_names_all_staff_pending_employment_cases = ['case_type_num', 'Staff Assigned To', 'Tracking Number', 'Case Type',  'Intake Date', 'Status', 'Work Days Between', 'External Agency', 'Date Response Submitted', 'Complainant Agency', 'Respondent Agency']

col_names_all_staff_weekly_pending_cases = ['case_type_num', 'Staff Assigned To', 'Tracking Number', 'Case Type',  'Intake Date', 'Status', 'Work Days Between', 'External Agency', 'Date Response Submitted', 'Complainant Agency', 'Respondent Agency', 'Region']

col_names_fns_pending_report = ['case_type_num', 'Staff Assigned To', 'Tracking Number', 'Case Type',  'Intake Date', 'Status', 'SNAPPER Issue', 'Date Sent to FNS', 'External Agency', 'Date Response Submitted', 'Work Days Between', 'Report Due to Program Coordinator', '90th Calendar Date']

#Define all the columns used in each report
col_names_open_charge_DEPT_pending =      ['Tracking Number', 'Status', 'case_type_num', 'Intake Date',  'Respondent Agency', 'External Agency']

col_names_open_charge_pending_ext_investgation = ['case_type_num', 'Tracking Number', 'How Received', 'Intake Date', 'Work Days Between',  'Respondent Agency', 'External Agency', 'Date Response Submitted', 'Days Since Response Submitted', 'Exteranl Agency Due Date', 'Staff Assigned To', 'Status']

col_names_pending_report_over_60_days = [ 'case_type_num', 'Staff Assigned To', 'Tracking Number', 'Case Type',  'Intake Date', 'Status', 'Work Days Between', 'External Agency', 'Date Response Submitted', 'Complainant Agency', 'Respondent Agency', 'Staff Region']

col_names_pending_report_over_60_days_minus_12_23_26_32_36 = ['case_type_num', 'Region', 'Staff Assigned To', 'Tracking Number', 'Case Type',  'Intake Date', 'Status', 'Work Days Between']


col_names_pending_report_over_60_days_minus_26s_31s = ['case_type_num', 'Staff Region', 'Staff Assigned To', 'Tracking Number', 'Case Type',  'Intake Date', 'Status', 'Work Days Between', 'External Agency', 'Date Response Submitted', 'Complainant Agency', 'Respondent Agency']


col_names_rar_pending_report = ['case_type_num', 'Staff Region', 'Staff Assigned To', 'Tracking Number', 'Case Type',  'Intake Date', 'Status', 'Work Days Between', 'Complainant Agency']


col_names_received_during_month = ['case_type_num', 'Staff Region', 'Staff Assigned To', 'Tracking Number', 'Case Type',  'Intake Date']


col_names_z_all_fns_pending_tracker = [ 'case_type_num', 'Staff Assigned To', 'Region', 'Tracking Number', 'Case Type',  'Intake Date', 'Status', 'SNAPPER Issue', 'Date Sent to FNS', 'Calendar Days', 'Report Due to Program Coordinator', '90th Calendar Date']

col_names_z_all_staff_pending_client_cases = ['case_type_num', 'Region', 'Staff Assigned To', 'Tracking Number', 'Case Type',  'Intake Date', 'Status', 'External Agency', 'Date Response Submitted', 'Work Days Between']


print('columns and names created')

columns and names created


In [283]:

AM_00_07_09 = ['00,07,09']
AM_08_10_11 = ['08,10,11']
AM_01_02_03 = ['01,02,03']
AM_04_05_06 = ['04,05,06']
COSTAFF = ['COSTAFF']
df['region_num'] = df['Region']

## Set Area Offices Based on names and regions

## NON FNS Report
* case types are 22, 26, 31, 34, 36
* closed date is NULL
* SNAPPER Issue is N (or No)
One sheet only

In [284]:
#Generate the dataframe
nonfns_pending_report = \
    df[(df['case_type_num'].isin(case_types_pending_nonfns_report)) &
       (df['SNAPPER Issue'] == 'N') &
       (df['Closed Date'].isnull())][col_names_pending_nonfns_report]

nonfns_pending_report.name = "\
                                All NON FNS pending report\
                                    "

# Renames not needed; cell will be commented out in case it is needed in the future

rlist = [nonfns_pending_report]

for report in rlist:
    report.drop_duplicates(inplace = True)
    report.sort_values(by=['Intake Date'], inplace = True, ascending = [True])
    report.drop(['case_type_num'], axis = 1, inplace = True)
    print(report.columns)

# Renames not needed

#for report in rlist:
#    report.rename(columns={'Staff Assigned To':'STAFF',
#                           'Tracking Number':'CASE NUM',
#                           'Case Type':'CASE TYPE',
#                           'Intake Date':'INTAKE DT',
#                           'Status':'CASE STATUS',
#                           'SNAPPER Issue':'SNAPPERS',
#                           'Date Sent to FNS':'DATE SENT TO FNS',
#                           'Report Due to Program Coordinator':'Report Due to Konstantina'}, inplace = True)

Index(['Staff Assigned To', 'Tracking Number', 'Case Type', 'Intake Date',
       'Status', 'SNAPPER Issue', 'Date Sent to FNS', 'External Agency',
       'Date Response Submitted'],
      dtype='object')


In [285]:
# Generate the dataframe
fns_pending_submitted_report = \
 df[(df['case_type_num'].isin(case_types_pending_fns_report)) &
 (df['Date Sent to FNS'].notnull()) &
 (df['SNAPPER Issue'] == 'Y') &
    (df['Closed Date'].isnull())][col_names_pending_fns_report]

fns_pending_submitted_report.name = "\
                                    All FNS Pending reports - Submitted\
                                    "

fns_pending_pending_report = \
 df[(df['case_type_num'].isin(case_types_pending_fns_report)) &
 (df['Date Sent to FNS'].isnull()) &
 (df['SNAPPER Issue'] == 'Y') & (df['Closed Date'].isnull())][col_names_pending_fns_report]

fns_pending_pending_report.name = "\
                                    All FNS Pending reports - Pending\
                                    "
# Rename columns as needed
rlist = [fns_pending_submitted_report, fns_pending_pending_report]

for report in rlist:
    report.drop_duplicates(inplace = True)
    report.sort_values(by=['Intake Date'], inplace = True, ascending = [True])
    report.rename(columns={'Staff Assigned To':'STAFF',
                           'Tracking Number':'CASE NUM',
                           'Case Type':'CASE TYPE',
                           'Intake Date':'INTAKE DT',
                           'Status':'CASE STATUS',
                           'SNAPPER Issue':'SNAPPERS',
                           'Date Sent to FNS':'DATE SENT TO FNS',
                           'Report Due to Program Coordinator':'Report Due'}, inplace = True)
    report.drop(['case_type_num'], axis = 1, inplace = True)
    print(report.columns)

Index(['STAFF', 'CASE NUM', 'CASE TYPE', 'INTAKE DT', 'CASE STATUS',
       'SNAPPERS', 'DATE SENT TO FNS', 'Calendar Days', 'Report Due',
       '90th Calendar Date'],
      dtype='object')
Index(['STAFF', 'CASE NUM', 'CASE TYPE', 'INTAKE DT', 'CASE STATUS',
       'SNAPPERS', 'DATE SENT TO FNS', 'Calendar Days', 'Report Due',
       '90th Calendar Date'],
      dtype='object')


In [286]:
#Generate the dataframe
all_past_due_rar_pending_report = \
    df[(df['case_type_num'].isin(case_types_all_past_due_rar_pending_report)) &
        (df['Work Days Between'] > 60) &
        (df['Closed Date'].isnull())][col_names_all_past_due_rar_pending_report]

all_past_due_rar_pending_report.name = "\
                                    All Past Due RAR Pending Report\
                                    "

#sort on intake date ascending
all_past_due_rar_pending_report.sort_values(by=['Work Days Between'], inplace=True, ascending=True)
# Rename all the columns in the list as needed
rlist = [all_past_due_rar_pending_report]

for report in rlist:
    report.drop_duplicates(inplace = True)
    report.sort_values(by=['Staff Assigned To', 'Intake Date'], inplace = True, ascending = [True, True])
    report.rename(columns={'Staff Assigned To':'STAFF',
                           'Tracking Number':'CASE NUM',
                           'Case Type':'CASE TYPE',
                           'Intake Date':'INTAKE DT',
                           'Status':'CASE STATUS',
                           'Work Days Between':'DAYS OPEN',
                           'Complainant Agency':'C-AGENCY'}, inplace = True)
        
    report.drop(['case_type_num'], axis = 1, inplace = True)
    print(report.columns)

Index(['STAFF', 'CASE NUM', 'CASE TYPE', 'INTAKE DT', 'CASE STATUS',
       'DAYS OPEN', 'C-AGENCY'],
      dtype='object')


In [287]:
#Generate the dataframe
all_staff_pending_admin_complaints_report = \
    df[(df['case_type_num'].isin(case_types_staff_pending_admin_complaints)) &
     (df['Closed Date'].isnull())]\
         [col_names_staff_pending_admin_complaints]

all_staff_pending_admin_complaints_report.name = "\
                                    All Staff Pending Administration Complaints\
                                    "

# rename columns as needed and confirm printed names
rlist = [all_staff_pending_admin_complaints_report]

for report in rlist:
    report.drop_duplicates(inplace = True)
    report.rename(columns={'Staff Assigned To':'STAFF',
                           'Tracking Number':'CASE NUM',
                           'Case Type':'CASE TYPE',
                           'Intake Date':'INTAKE DT',
                           'Status':'CASE STATUS',
                          'External Agency':'EXTNL AGENCY',
                          'Date Response Submitted':'DT RESPONSE',
                           'Complainant Agency':'C-AGENCY',
                           'Respondent Agency':'R-AGENCY',
                          }, inplace = True)
        
    report.drop(['case_type_num'], axis = 1, inplace = True)
    print(report.columns)

Index(['STAFF', 'CASE NUM', 'CASE TYPE', 'INTAKE DT', 'CASE STATUS',
       'Work Days Between', 'EXTNL AGENCY', 'DT RESPONSE', 'C-AGENCY',
       'R-AGENCY'],
      dtype='object')


In [288]:
#Generate the dataframe
all_staff_pending_client_cases_report = \
    df[(df['case_type_num'].isin(case_types_all_staff_pending_client_cases)) &
     (df['Closed Date'].isnull())]\
         [col_names_all_staff_pending_client_cases]

In [289]:
#Generate the dataframe
all_staff_pending_edcs_for_ops_program_mgr_report =\
     df[\
        (df['case_type_num'].isin(case_types_all_staff_pending_edcs_for_ops_program_mgr))&
         (df['Closed Date'].isnull())]\
         [col_names_all_staff_pending_edcs_for_ops_program_mgr]

all_staff_pending_edcs_for_ops_program_mgr_report.name = "\
                                    All Staff Pending EDCs for Ops Program Manager\
                                    "
# rename columns as needed and confirm printed names
rlist = [all_staff_pending_edcs_for_ops_program_mgr_report]

for report in rlist:
    report.drop_duplicates(inplace = True)
    report.drop(['case_type_num'], axis = 1, inplace = True)
    report.sort_values(by=['Case Type', 'Tracking Number'], inplace = True, ascending = [True, True])
    print(report.columns)

Index(['Case Type', 'Tracking Number', 'Intake Date', 'Region',
       'Staff Assigned To', 'Status'],
      dtype='object')


In [290]:
#Generate the dataframe
all_staff_pending_employment_cases_report = \
    df[(df['case_type_num'].isin(case_types_all_staff_pending_employment_cases))&
         (df['Closed Date'].isnull())]\
         [col_names_all_staff_pending_employment_cases]

all_staff_pending_employment_cases_report.name = "\
                                    All Staff Pending Employment Cases\
                                    "
# rename columns as needed and confirm printed names
rlist = [all_staff_pending_employment_cases_report]
for report in rlist:
    report.drop_duplicates(inplace = True)
    report.drop(['case_type_num'], axis = 1, inplace = True)
    report.sort_values(by=['Staff Assigned To', 'Tracking Number'], inplace = True, ascending = [True, True])
    print(report.columns)
    
all_staff_pending_employment_cases_report.info()

Index(['Staff Assigned To', 'Tracking Number', 'Case Type', 'Intake Date',
       'Status', 'Work Days Between', 'External Agency',
       'Date Response Submitted', 'Complainant Agency', 'Respondent Agency'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 38 entries, 352 to 375
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Staff Assigned To        38 non-null     object        
 1   Tracking Number          38 non-null     object        
 2   Case Type                38 non-null     object        
 3   Intake Date              38 non-null     datetime64[ns]
 4   Status                   38 non-null     category      
 5   Work Days Between        38 non-null     float64       
 6   External Agency          0 non-null      object        
 7   Date Response Submitted  0 non-null      datetime64[ns]
 8   Complainant Agency       38 non-null     object

### Generate the DataFrame

In [291]:
#Generate the dataframe
all_staff_weekly_pending_cases_report =\
     df[(df['case_type_num'].isin(case_types_all_staff_weekly_pending_cases)) &
      (df['Closed Date'].isnull())]\
          [col_names_all_staff_weekly_pending_cases]

all_staff_weekly_pending_cases_report.sort_values(by='Work Days Between', inplace = True, ascending=False)


all_staff_weekly_pending_cases_report.name = "\
                                    All Staff Weekly Pending Cases\
                                    "

# rename columns as needed and confirm printed names
rlist = [all_staff_weekly_pending_cases_report]

for report in rlist:
    report.drop_duplicates(inplace = True)
    report.rename(columns={'Staff Assigned To':'STAFF',
                           'Tracking Number':'CASE NUM',
                           'Case Type':'CASE TYPE',
                           'Intake Date':'INTAKE DT',
                           'Status':'CASE STATUS',
                          'External Agency':'EXTNL AGENCY',
                          'Date Response Submitted':'DT RESPONSE',
                           'Complainant Agency':'C-AGENCY',
                           'Respondent Agency':'R-AGENCY',
                          }, inplace = True)
    report.sort_values(by=['Work Days Between', 'STAFF'], inplace = True, ascending = [False, True])    
    report.drop(['case_type_num'], axis = 1, inplace = True)
    print(report.columns)

Index(['STAFF', 'CASE NUM', 'CASE TYPE', 'INTAKE DT', 'CASE STATUS',
       'Work Days Between', 'EXTNL AGENCY', 'DT RESPONSE', 'C-AGENCY',
       'R-AGENCY', 'Region'],
      dtype='object')


### Generate the DataFrame

In [292]:
df.columns

Index(['Type', 'Tracking Number', 'Case Type', 'Intake Date', 'Closed Date',
       'Finding', 'Result', 'Respondent Agency', 'Complainant Agency',
       'Received By', 'Staff Assigned To', 'Mediation Requested', 'Region',
       'No basis identified', 'SNAPPER Issue', 'Status',
       'Mediation Start Date', 'Mediation End Date', 'LEP', 'Complainant City',
       'Respondent First Name', 'Respondent Race', 'Respondent Sex',
       'Respondent City', 'External Agency', 'Client Type',
       'Date Response Submitted', 'Date Sent to FNS',
       'Exteranl Agency Due Date', 'Created Date', 'DUE Date', 'Date Assigned',
       'FNS Concurrence Date', 'How Received', 'today', 'SNAPPER',
       'case_type_num', 'case_type_desc', '90th Calendar Date',
       'Report Due to Program Coordinator', 'Calendar Days',
       'Days Since Response Submitted', 'Work Days Between',
       '90 Business Days', '90 Business Days minus 1', 'nb_months',
       'region_num'],
      dtype='object')

In [293]:
#Generate the dataframe
#central regions
fns_pending_report_AM_00_07_09_report = \
    df[(df['case_type_num'].isin(case_types_fns_pend)) &
     (df['SNAPPER Issue'] == 'Y') &
      (df['Closed Date'].isnull()) &
       (df['Staff Region'].isin(AM_00_07_09))]\
           [col_names_fns_pending_report]

fns_pending_report_AM_00_07_09_report.name = "\
                                    FNS Pending Report Region 00, 07 09\
                                    "

fns_pending_report_AM_01_02_03_report =\
     df[(df['case_type_num'].isin(case_types_fns_pend)) &
      (df['SNAPPER Issue'] == 'Y') &
       (df['Closed Date'].isnull()) &
        (df['Staff Region'].isin(AM_01_02_03))]\
            [col_names_fns_pending_report]

fns_pending_report_AM_01_02_03_report.name = "\
                                    FNS Pending Report Region 01, 02 03\
                                    "

fns_pending_report_AM_04_05_06_report =\
     df[(df['case_type_num'].isin(case_types_fns_pend)) &
      (df['SNAPPER Issue'] == 'Y') &
       (df['Closed Date'].isnull()) &
       (df['Staff Region'].isin(AM_04_05_06))]\
           [col_names_fns_pending_report]

fns_pending_report_AM_04_05_06_report.name = "\
                                    FNS Pending Report Region 04, 05 06\
                                    "

fns_pending_report_AM_08_10_11_report =\
     df[(df['case_type_num'].isin(case_types_fns_pend)) &
      (df['SNAPPER Issue'] == 'Y') &
       (df['Closed Date'].isnull()) &
        (df['Staff Region'].isin(AM_08_10_11))]\
         [col_names_fns_pending_report]
fns_pending_report_AM_08_10_11_report.name = "\
                                    FNS Pending Report Region 08, 10 11\
                                    "

fns_pending_report_AM_COSTAFF_report =\
     df[(df['case_type_num'].isin(case_types_fns_pend)) &
      (df['SNAPPER Issue'] == 'Y') &
       (df['Closed Date'].isnull()) &
        (df['Staff Assigned To'].isin(COSTAFF))]\
            [col_names_fns_pending_report]

fns_pending_report_AM_COSTAFF_report.name = "\
                                    FNS Pending Report COSTAFF\
                                    "


# rename columns as needed and confirm printed names
rlist = [fns_pending_report_AM_00_07_09_report, fns_pending_report_AM_01_02_03_report, fns_pending_report_AM_04_05_06_report, fns_pending_report_AM_08_10_11_report, fns_pending_report_AM_COSTAFF_report]


for report in rlist:
    report.drop_duplicates(inplace = True)
    report.rename(columns={'Staff Assigned To':'STAFF',
                           'Tracking Number':'CASE NUM',
                           'Case Type':'CASE TYPE',
                           'Intake Date':'INTAKE DT',
                           'Status':'CASE STATUS',
                           'SNAPPER Issue':'SNAPPERS',
                           'Date Sent to FNS':'DATE SENT TO FNS',
                           'External Agency': 'EXTNL AGENCY', 
                           'Date Response Submitted': 'DT RSPN SUBMITTED',
                           'Work Days Between': 'Calender Days',
                           'Report Due to Program Coordinator':'Report Due to Konstantina'}, inplace = True)
    
    report.sort_values(by=['STAFF', 'INTAKE DT'], inplace = True, ascending = [True, True])
    report.drop(['case_type_num'], axis = 1, inplace = True)

KeyError: 'Staff Region'

In [333]:
#Generate the dataframe
open_charge_DARS_pending_report = df \
[(df['case_type_num'].isin(case_types_open_charge)) &
 (df['Respondent Agency']== 'DARS') &
(df['Status'].str.contains("(?i)open|response submitted"))]\
    [col_names_open_charge_DEPT_pending].drop_duplicates(subset = ['Tracking Number'])

open_charge_CADS_pending_report.name = "\
                                    Open Charges Pending - CADS\
                                    "

open_charge_DONS_pending_report = df \
[(df['case_type_num'].isin(case_types_open_charge)) &
 (df['Respondent Agency']== 'DONS') &
(df['Status'].str.contains("(?i)open|response submitted"))]\
    [col_names_open_charge_DEPT_pending].drop_duplicates(subset = ['Tracking Number'])

open_charge_DONS_pending_report.name = "\
                                    Open Charges Pending - DONS\
                                    "


open_charge_ZIMZ_CADS_pending_report = df \
[(df['case_type_num'].isin(case_types_open_charge)) &
(df['Respondent Agency'].isin(['ZIMZ', 'CAPS']))  &
(df['Status'].str.contains("(?i)open|response submitted"))]\
    [col_names_open_charge_DEPT_pending].drop_duplicates(subset = ['Tracking Number'])


open_charge_ZIMZ_CADS_pending_report.name = "\
                                    Open Charges Pending - CAPS\
                                    "

# Generate new column names as needed
rlist = [open_charge_DFPS_pending_report, open_charge_DSHS_pending_report, open_charge_HHSC_DADS_pending_report]


In [324]:
#Generate the dataframe
open_charge_ext_pending_report = df \
[(df['case_type_num'].isin(case_types_open_charge)) &
(df['Status'].str.lower().str.match('o.*|r.*'))]\
    [col_names_open_charge_pending_ext_investgation]

open_charge_ext_pending_report.name = "\
                                    Open Charges Pending - External Agency\
                                    "

#sort on intake date ascending
open_charge_ext_pending_report.sort_values(by=['Respondent Agency'], inplace=True, ascending=True)

rlist = [open_charge_ext_pending_report]

for report in rlist:
    report.drop_duplicates(subset = ['Tracking Number'], inplace = True)
    report.rename(columns={'Tracking Number':'CRO Tracking Number',
                           'How Received':'How Received?',
                           'Intake Date':'Date Received+B2:B42',
                           'WorkDaysBetween':'# of Workdays Since Date Received',
                           'Respondent Agency':'Respondent Agency',
                           'Respondent First Name':'Respondent First Name','External Agency':'External Agency',
                           'Date Response Submitted':'Date Response submitted to External Agency',
                           'Days Since Response Submitted':'Calendar Days Since Response Submitted',
                           'Exteranl Agency Due Date':'Pending Response Due Date',
                           'Staff Assigned To':'CRO Staff Assigned',
                           'Status':'CRO Status Update/Comments'}, inplace = True)
    
    report.drop(['case_type_num'], axis = 1, inplace = True)
    display(report['CRO Tracking Number'])

23     Q04042016.0200003
25     Q08192016.0200001
20     Q03172015.0200003
260    Q09112020.0200008
266    Q09112020.0200007
             ...        
277    Q12222020.0200002
283    Q01072021.0200010
285    Q04092021.0200007
241    Q05022019.0200011
263    Q12152020.0200001
Name: CRO Tracking Number, Length: 71, dtype: object

### Column rename

In [325]:
#Generate the dataframe
Pending_report_over_60_days_minus_12_23_26_32_36 = \
    df[(df['case_type_num'].isin(case_types_pending_report_over_60_days_minus_12_23_26_32_36)) &
    (df['Work Days Between'] >= 50) &
    (df['Status']=='Open')]\
    [col_names_pending_report_over_60_days_minus_12_23_26_32_36]


Pending_report_over_60_days_minus_12_23_26_32_36.name = "\
                                    Pending Report Over 50 Days - minus 12,23,26,32,36\
                                    "
# Rename the columns
rlist = [Pending_report_over_60_days_minus_12_23_26_32_36]

for report in rlist:
    report.drop_duplicates(inplace = True)
    report.sort_values(by=['Region','Staff Assigned To'], inplace=True, ascending=True)
    report.rename(columns={'Tracking Number':'CASE NUM',
                           'Intake Date':'INTAKE DT',
                           'Case Type': 'CASE TYPE',
                           'WorkDaysBetween':'Work Days Between',
                           'Complainant Agency':'C-AGENCY',
                           'Respondent Agency':'R-AGENCY',
                           'Respondent First Name':'R-FIRST NAME',
                           'External Agency':'EXTNL AGENCY',
                           'External Case Number':'External Charge #',
                           'Date Response Submitted':'DT RSPN SUBMITTED',
                           'Days Since Response Submitted':'Calendar Days Since Response Submitted',
                           'Exteranl Agency Due Date':'Pending Response Due Date',
                           'Staff Assigned To':'STAFF',
                           'Status':'CASE STATUS'}, inplace = True)
    report.drop(['case_type_num'], axis=1, inplace= True)
    
                                    

### Generate the DataFrame

In [326]:
from datetime import *; from dateutil.relativedelta import *
import calendar
TODAY = date1
LAST_MONTH = TODAY+relativedelta(months=-1)

after_or_on = LAST_MONTH+relativedelta(day = 31, weekday=MO(-1))
print(after_or_on)
print(date1)

df['after_or_on'] = pd.to_datetime(after_or_on)

2021-04-26
2021-05-04


In [327]:
all_staff_pending_admin_complaints_report.columns

Index(['Type', 'Tracking Number', 'Case Type', 'Intake Date', 'Closed Date',
       'Finding', 'Result', 'Respondent Agency', 'Complainant Agency',
       'Received By', 'Staff Assigned To', 'Mediation Requested', 'Region',
       'No basis identified', 'SNAPPER Issue', 'Status',
       'Mediation Start Date', 'Mediation End Date', 'LEP', 'Complainant City',
       'Respondent First Name', 'Respondent Race', 'Respondent Sex',
       'Respondent City', 'External Agency', 'Client Type',
       'Date Response Submitted', 'Date Sent to FNS',
       'Exteranl Agency Due Date', 'Created Date', 'DUE Date', 'Date Assigned',
       'FNS Concurrence Date', 'How Received', 'today', 'SNAPPER',
       'case_type_num', 'case_type_desc', '90th Calendar Date',
       'Report Due to Program Coordinator', 'Calendar Days',
       'Days Since Response Submitted', 'Work Days Between',
       '90 Business Days', '90 Business Days minus 1', 'nb_months',
       'region_num', 'after_or_on'],
      dtype='objec

In [328]:
all_staff_pending_admin_complaints_report[['EXTNL AGENCY', 'DT RESPONSE']] = all_staff_pending_admin_complaints_report[['EXTNL AGENCY', 'DT RESPONSE']].fillna("")
all_staff_pending_admin_complaints_report.reset_index(inplace = True)

KeyError: "None of [Index(['EXTNL AGENCY', 'DT RESPONSE'], dtype='object')] are in the [columns]"

In [329]:
all_staff_pending_admin_complaints_report = \
    df[(df['case_type_num'].isin(case_types_staff_pending_admin_complaints)) &
     (df['Closed Date'].isnull())]\
         [col_names_staff_pending_admin_complaints]

In [330]:
from IPython.display import HTML
from jinja2 import Environment, FileSystemLoader

for staff in s_list:
    all_staff_pending_admin_complaints_report = \
            df[(df['Staff Assigned To'] == staff)]
    all_staff_pending_admin_complaints_report.name = "Pending Cases for "+ staff
    #all_staff_pending_admin_complaints_report.to_html('all_staff_pending_admin_complaints_report_' +staff+ '.html')
    env = Environment(loader=FileSystemLoader('.'))
    template = env.get_template("mytemplate.html")
    
    env = Environment(loader=FileSystemLoader('.'))
    template = env.get_template("mytemplate.html")
    
    template_vars = {"title" : all_staff_pending_admin_complaints_report.name + date1.strftime("%m-%d-%Y"),
                 "national_pivot_table": all_staff_pending_admin_complaints_report.to_html(),
                    "header2" : str(df['Region'].unique())}
    html_out = template.render(template_vars)
    
    # to save the results
    with open("All Staff Pending Admin Complaints " + staff + ".html", "w") as fh:
        fh.write(html_out)

# Create The Excel Files

## Create Excel Files for Single Worksheet Reports



### Repeated SIngle Sheet Reports with no tab names

In [335]:
# Generate a list of reports with expected names to be created in single sheets
list_reports = {'All Non-Pending Cases as of ':nonfns_pending_report,
'Weekly Pending Complaints as of ':all_staff_pending_admin_complaints_report,
'All Staff Weekly Pending Cases as of ':all_staff_weekly_pending_cases_report,
'All Staff Pending Program Mgr as of ': all_staff_pending_edcs_for_ops_program_mgr_report,
'All Staff Pending Employment Cases as of ':all_staff_pending_employment_cases_report}




    #'z_tracker_1':z_all_FNS_pending_tracker_report,
    #'z_tracker_2':z_all_staff_pending_client_cases_report}

# Create a Pandas Excel writer using XlsxWriter as the engine.
import pandas.io.formats.excel
pandas.io.formats.excel.ExcelFormatter.header_style = None

for filename, codename in list_reports.items():
    date_str = date1.strftime("%m-%d-%Y")
    extension = '.xlsx'
    filename = filename+date_str+extension
    writer = pd.ExcelWriter(filename, engine='xlsxwriter', datetime_format='mm/dd/yyyy',
                            date_format='mm/dd/yyyy')

    # Write each dataframe to a different worksheet.
    codename.to_excel(writer, sheet_name= 'Sheet 1', index = False)


    workbook  = writer.book


    font_fmt = workbook.add_format({'font_name': 'Calibri', 'font_size': 10, 'text_wrap': True})
    header_fmt = workbook.add_format({'font_name': 'Times New Roman', 'font_size': 10, 'fg_color': '#BFBFBF', 'text_wrap': True})


    worksheet = writer.sheets['Sheet 1']
    
    # Freeze pane on the top row.
    worksheet.freeze_panes(1, 0)

    
    worksheet.set_column('A:A', 25, font_fmt)
    worksheet.set_column('B:B', 18, font_fmt)
    worksheet.set_column('C:C', 35, font_fmt)
    worksheet.set_column('D:F', 15, font_fmt)
    worksheet.set_column('G:G', 40, font_fmt)
    worksheet.set_column('H:O', 17, font_fmt)
    worksheet.set_row(0, 29, header_fmt)


    # Close the Pandas Excel writer and output the Excel file.
    writer.save()
    print('Report created:' + filename)
    
list_reports = {'All Past Due PSP Pending Cases as of ':all_past_due_rar_pending_report}
# Create a Pandas Excel writer using XlsxWriter as the engine.
import pandas.io.formats.excel
pandas.io.formats.excel.ExcelFormatter.header_style = None

for filename, codename in list_reports.items():
    date_str = date1.strftime("%m-%d-%Y")
    extension = '.xlsx'
    filename = filename+date_str+extension
    writer = pd.ExcelWriter(filename, engine='xlsxwriter', datetime_format='mm/dd/yyyy',
                            date_format='mm/dd/yyyy')

    # Write each dataframe to a different worksheet.
    codename.to_excel(writer, sheet_name= 'All Past Due RAR Pending Cases', index = False)


    workbook  = writer.book


    font_fmt = workbook.add_format({'font_name': 'Calibri', 'font_size': 10, 'text_wrap': True})
    header_fmt = workbook.add_format({'font_name': 'Times New Roman', 'font_size': 10, 'fg_color': '#BFBFBF', 'text_wrap': True})


    worksheet = writer.sheets['All Past Due RAR Pending Cases']
    
    # Freeze pane on the top row.
    worksheet.freeze_panes(1, 0)

    worksheet.set_column('A:A', 25, font_fmt)
    worksheet.set_column('B:B', 18, font_fmt)
    worksheet.set_column('C:C', 35, font_fmt)
    worksheet.set_column('D:F', 15, font_fmt)
    worksheet.set_column('G:G', 40, font_fmt)
    worksheet.set_column('H:O', 17, font_fmt)
    worksheet.set_row(0, 29, header_fmt)


    # Close the Pandas Excel writer and output the Excel file.
    writer.save()
    print('Report created:' + filename)
    
# Create a Pandas Excel writer using XlsxWriter as the engine.
import pandas.io.formats.excel
pandas.io.formats.excel.ExcelFormatter.header_style = None


date_str = date1.strftime("%m-%d-%Y")
extension = '.xlsx'
filename = 'Pending Cases Over 60 Days as of '+date_str+ ' without 26s & 31s'+extension
writer = pd.ExcelWriter(filename, engine='xlsxwriter', datetime_format='mm/dd/yyyy',
                            date_format='mm/dd/yyyy')

# Write each dataframe to a different worksheet.
Pending_report_over_60_days_minus_26s_31s.to_excel(writer, sheet_name= 'Over 60 Days without 26s & 31s', index = False)


workbook  = writer.book


font_fmt = workbook.add_format({'font_name': 'Calibri', 'font_size': 10, 'text_wrap': True})
header_fmt = workbook.add_format({'font_name': 'Times New Roman', 'font_size': 10, 'fg_color': '#BFBFBF', 'text_wrap': True})


worksheet = writer.sheets['Over 60 Days without 26s & 31s']

# Freeze pane on the top row.
worksheet.freeze_panes(1, 0)

worksheet.set_column('A:A', 25, font_fmt)
worksheet.set_column('B:B', 18, font_fmt)
worksheet.set_column('C:C', 35, font_fmt)
worksheet.set_column('D:F', 15, font_fmt)
worksheet.set_column('G:G', 40, font_fmt)
worksheet.set_column('H:O', 17, font_fmt)
worksheet.set_row(0, 29, header_fmt)


# Close the Pandas Excel writer and output the Excel file.
writer.save()

print('Report Created: ' + filename)


# Create a Pandas Excel writer using XlsxWriter as the engine.
import pandas.io.formats.excel
pandas.io.formats.excel.ExcelFormatter.header_style = None


date_str = date1.strftime("%m-%d-%Y")
extension = '.xlsx'
filename = 'Pending Cases Over 50 Days as of '+date_str+ ' without 12-23-26-32-36'+extension
writer = pd.ExcelWriter(filename, engine='xlsxwriter', datetime_format='mm/dd/yyyy',
                            date_format='mm/dd/yyyy')

# Write each dataframe to a different worksheet.
Pending_report_over_60_days_minus_12_23_26_32_36.to_excel(writer, sheet_name= 'Over 50 Days (-) 12-23-26-32-36', index = False)


workbook  = writer.book


font_fmt = workbook.add_format({'font_name': 'Calibri', 'font_size': 10, 'text_wrap': True})
header_fmt = workbook.add_format({'font_name': 'Times New Roman', 'font_size': 10, 'fg_color': '#BFBFBF', 'text_wrap': True})


worksheet = writer.sheets['Over 50 Days (-) 12-23-26-32-36']
# Freeze pane on the top row.
worksheet.freeze_panes(1, 0)

worksheet.set_column('A:A', 25, font_fmt)
worksheet.set_column('B:B', 18, font_fmt)
worksheet.set_column('C:C', 20, font_fmt)
worksheet.set_column('D:D', 47, font_fmt)
worksheet.set_column('E:G', 10, font_fmt)
worksheet.set_column('H:H', 7, font_fmt)
worksheet.set_row(0, 29, header_fmt)


    # Close the Pandas Excel writer and output the Excel file.
writer.save()

print('Report Created: ' + filename)

# Create a Pandas Excel writer using XlsxWriter as the engine.
import pandas.io.formats.excel
pandas.io.formats.excel.ExcelFormatter.header_style = None

date_str = date1.strftime("%m-%d-%Y")
extension = '.xlsx'
filename = "All FNS Pending Cases as of "+date_str+extension
writer = pd.ExcelWriter(filename, engine='xlsxwriter', datetime_format='mm/dd/yyyy',
                        date_format='mm/dd/yyyy')

# Write each dataframe to a different worksheet.

fns_pending_pending_report.to_excel(writer, sheet_name= 'FNS Pending', index = False)
fns_pending_submitted_report.to_excel(writer, sheet_name= 'FNS Submitted', index = False)

workbook  = writer.book
worksheet = writer.sheets['FNS Pending']

font_fmt = workbook.add_format({'font_name': 'Calibri', 'font_size': 10, 'text_wrap': True})
header_fmt = workbook.add_format({'font_name': 'Times New Roman', 'font_size': 10, 'fg_color': '#BFBFBF', 'text_wrap': True})


worksheet = writer.sheets['FNS Pending']
# Freeze pane on the top row.
worksheet.freeze_panes(1, 0)


worksheet.set_column('A:A', 16, font_fmt)
worksheet.set_column('B:B', 18, font_fmt)
worksheet.set_column('C:C', 35, font_fmt)
worksheet.set_column('D:F', 15, font_fmt)
worksheet.set_column('G:I', 12, font_fmt)
worksheet.set_column('J:J', 16, font_fmt)
worksheet.set_column('K:M', 12, font_fmt)
worksheet.set_row(0, 29, header_fmt)

worksheet = writer.sheets['FNS Submitted']
# Freeze pane on the top row.
worksheet.freeze_panes(1, 0)


worksheet.set_column('A:A', 16, font_fmt)
worksheet.set_column('B:B', 17, font_fmt)
worksheet.set_column('C:C', 33, font_fmt)
worksheet.set_column('D:E', 18, font_fmt)
worksheet.set_column('F:F', 12, font_fmt)
worksheet.set_column('G:G', 44, font_fmt)
worksheet.set_column('H:H', 6.5, font_fmt)
worksheet.set_column('I:I', 12, font_fmt)
worksheet.set_column('J:J', 16, font_fmt)
worksheet.set_column('K:M', 12, font_fmt)
worksheet.set_row(0, 29, header_fmt)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

print('Report Created: ' + filename)

# Create a Pandas Excel writer using XlsxWriter as the engine.
fns_regional_list = {'Reg 00-07-09 ':fns_pending_report_AM_00_07_09_report,
                     'Reg 01-02-03 ':fns_pending_report_AM_01_02_03_report,
                     'Reg 04-05-06 ':fns_pending_report_AM_04_05_06_report,
                     'Reg 08-10-11 ':fns_pending_report_AM_08_10_11_report,
                     'Reg CO Staff ':fns_pending_report_AM_COSTAFF_report}

for filename, codename in fns_regional_list.items():
    date_str = date1.strftime("%m-%d-%Y")
    extension = '.xlsx'
    file = 'FNS Pending Cases '+filename +'as of '+date_str +extension
    writer = pd.ExcelWriter(file, engine='xlsxwriter', datetime_format='mm/dd/yyyy',
                            date_format='mm/dd/yyyy')

    # Write each dataframe to a different worksheet.

    codename.to_excel(writer, sheet_name= filename, index = False)

    workbook  = writer.book
    worksheet = writer.sheets[filename]
    worksheet.freeze_panes(1,0)


    font_fmt = workbook.add_format({'font_name': 'Calibri',
                                    'font_size': 10,
                                    'text_wrap': True})
   
    header_fmt = workbook.add_format({'font_name': 'Times New Roman',
                                      'font_size': 10,
                                      'fg_color': '#BFBFBF',
                                      'text_wrap': True})

    # Close the Pandas Excel writer and output the Excel file.
    writer.save()
    
    print( 'Report Created: '+ file)
    
# Create a Pandas Excel writer using XlsxWriter as the engine.
import pandas.io.formats.excel
pandas.io.formats.excel.ExcelFormatter.header_style = None

date_str = date1.strftime("%m-%d-%Y")
extension = '.xlsx'
filename = "Pending Cases Over 60 Days as of "+date_str+extension
writer = pd.ExcelWriter(filename, engine='xlsxwriter', datetime_format='mm/dd/yyyy',
                        date_format='mm/dd/yyyy')

# Write each dataframe to a different worksheet.

pending_report_over_60_days_AM_00_07_09_report.to_excel(writer, sheet_name= 'Region 00, 07 09', index = False)
pending_report_over_60_days_AM_01_02_03_report.to_excel(writer, sheet_name= 'Region 01, 02 03', index = False)
pending_report_over_60_days_AM_04_05_06_report.to_excel(writer, sheet_name= 'Region 04, 05 06', index = False)
pending_report_over_60_days_AM_08_10_11_report.to_excel(writer, sheet_name= 'Region 08, 10 11', index = False)
pending_report_over_60_days_AM_COSTAFF_report.to_excel(writer, sheet_name= 'CO Staff', index = False)

workbook  = writer.book


font_fmt = workbook.add_format({'font_name': 'Calibri', 'font_size': 10, 'text_wrap': True})
header_fmt = workbook.add_format({'font_name': 'Times New Roman', 'font_size': 10, 'fg_color': '#BFBFBF', 'text_wrap': True})

region_list = ['Region 00, 07 09', 'Region 01, 02 03', 'Region 04, 05 06', 'Region 08, 10 11', 'CO Staff']

for region in region_list:
    worksheet = writer.sheets[region]
    worksheet.freeze_panes(1,0)
    worksheet.set_column('A:A', 20, font_fmt)
    worksheet.set_column('B:B', 18, font_fmt)
    worksheet.set_column('C:C', 45, font_fmt)
    worksheet.set_column('D:F', 15, font_fmt)
    worksheet.set_column('G:G', 45, font_fmt)
    worksheet.set_column('H:H', 6.5, font_fmt)
    worksheet.set_column('I:K', 12, font_fmt)
    worksheet.set_column('L:L', 6.5, font_fmt)
    worksheet.set_column('M:N', 12, font_fmt)
    worksheet.set_column('O:O', 6.5, font_fmt)
    worksheet.set_row(0, 29, header_fmt)


# Close the Pandas Excel writer and output the Excel file.
writer.save()

print('Report Created: ' + filename)


# Create a Pandas Excel writer using XlsxWriter as the engine.
import pandas.io.formats.excel
pandas.io.formats.excel.ExcelFormatter.header_style = None

date_str = date1.strftime("%m-%d-%Y")
extension = '.xlsx'
filename = "Pending RAR Cases as of "+date_str+extension
writer = pd.ExcelWriter(filename, engine='xlsxwriter', datetime_format='mm/dd/yyyy',
                        date_format='mm/dd/yyyy')

# Write each dataframe to a different worksheet.

rar_pending_report_AM_00_07_09_report.to_excel(writer, sheet_name= 'Region 00, 07 09', index = False)
rar_pending_report_AM_01_02_03_report.to_excel(writer, sheet_name= 'Region 01, 02 03', index = False)
rar_pending_report_AM_04_05_06_report.to_excel(writer, sheet_name= 'Region 04, 05 06', index = False)
rar_pending_report_AM_08_10_11_report.to_excel(writer, sheet_name= 'Region 08, 10 11', index = False)
rar_pending_report_AM_COSTAFF_report.to_excel(writer, sheet_name= 'CO Staff', index = False)

workbook  = writer.book


font_fmt = workbook.add_format({'font_name': 'Calibri', 'font_size': 10, 'text_wrap': True})
header_fmt = workbook.add_format({'font_name': 'Times New Roman', 'font_size': 10, 'fg_color': '#BFBFBF', 'text_wrap': True})

region_list = ['Region 00, 07 09', 'Region 01, 02 03', 'Region 04, 05 06', 'Region 08, 10 11', 'CO Staff']

for region in region_list:
    worksheet = writer.sheets[region]
    worksheet.freeze_panes(1,0)
    worksheet.set_column('A:A', 25, font_fmt)
    worksheet.set_column('B:B', 18, font_fmt)
    worksheet.set_column('C:C', 35, font_fmt)
    worksheet.set_column('D:F', 15, font_fmt)
    worksheet.set_column('G:K', 17, font_fmt)
    worksheet.set_row(0, 29, header_fmt)


# Close the Pandas Excel writer and output the Excel file.
writer.save()

print('Report Created: ' + filename)

# Create a Pandas Excel writer using XlsxWriter as the engine.
import pandas.io.formats.excel
pandas.io.formats.excel.ExcelFormatter.header_style = None

date_str = date1.strftime("%m-%d-%Y")
from_date = after_or_on.strftime("%m-%d-%Y")
extension = '.xlsx'
filename = "Complaints Received During Month From " + from_date+  " as of "+date_str+extension
writer = pd.ExcelWriter(filename, engine='xlsxwriter', datetime_format='mm/dd/yyyy',
                        date_format='mm/dd/yyyy')

# Write each dataframe to a different worksheet.
received_during_month_AM_00_07_09_report.to_excel(writer, sheet_name= 'Region 00, 07 09', index = False)
received_during_month_AM_01_02_03_report.to_excel(writer, sheet_name= 'Region 01, 02 03', index = False)
received_during_month_AM_04_05_06_report.to_excel(writer, sheet_name= 'Region 04, 05 06', index = False)
received_during_month_AM_08_10_11_report.to_excel(writer, sheet_name= 'Region 08, 10 11', index = False)
received_during_month_AM_COSTAFF_report.to_excel(writer, sheet_name= 'CO Staff', index = False)

workbook  = writer.book


font_fmt = workbook.add_format({'font_name': 'Calibri', 'font_size': 10, 'text_wrap': True})
header_fmt = workbook.add_format({'font_name': 'Times New Roman', 'font_size': 10, 'fg_color': '#BFBFBF', 'text_wrap': True})

region_list = ['Region 00, 07 09', 'Region 01, 02 03', 'Region 04, 05 06', 'Region 08, 10 11', 'CO Staff']

for region in region_list:
    worksheet = writer.sheets[region]
    worksheet.freeze_panes(1,0)
    
    worksheet.set_column('A:A', 25, font_fmt)
    worksheet.set_column('B:B', 18, font_fmt)
    worksheet.set_column('C:C', 35, font_fmt)
    worksheet.set_column('D:F', 15, font_fmt)
    worksheet.set_column('G:G', 40, font_fmt)
    worksheet.set_column('H:O', 17, font_fmt)
    worksheet.set_row(0, 29, header_fmt)


# Close the Pandas Excel writer and output the Excel file.
writer.save()

print('Report Created: ' + filename)

# Create a Pandas Excel writer using XlsxWriter as the engine.
import pandas.io.formats.excel
pandas.io.formats.excel.ExcelFormatter.header_style = None

date_str = date1.strftime("%m-%d-%Y")
extension = '.xlsx'
filename = "Pending Charges Report as of "+date_str+ ' - ZIMZ & CADS, DONS, DARS'+extension
writer = pd.ExcelWriter(filename, engine='xlsxwriter', datetime_format='mm/dd/yyyy',
                        date_format='mm/dd/yyyy')

# Write each dataframe to a different worksheet.

open_charge_DONS_pending_report.to_excel(writer, sheet_name= 'DONS', index = False, startrow = 4)
open_charge_DARS_pending_report.to_excel(writer, sheet_name= 'DARS', index = False,  startrow = 4)
open_charge_ZIMZ_CADS_pending_report.to_excel(writer, sheet_name= 'ZIMZ & CADS', index = False,  startrow = 4)


workbook  = writer.book
merge_format = workbook.add_format({
    'bold': True,
    'font_name': 'Ariel',
    'font_size': 14,
    'align': 'center',
    'valign': 'vcenter'})
font_fmt = workbook.add_format({
    'font_name': 'Calibri',
    'font_size': 10,
    'text_wrap': True})
header_fmt = workbook.add_format({
    'font_name': 'Ariel',
    'bold':True,
    'border': 1,
    'font_size': 9,
    'fg_color': '#BFBFBF',
    'text_wrap': True,
    'align': 'center',
    'valign': 'vcenter'})

department_list = {'DONS':'Don''s Issues', 'DARS':'David''s Services', 'ZIMZ & CADS': 'Zim and Cade''s Garage'}

for department, deparment_name in department_list.items():
    worksheet = writer.sheets[department]
    
    # Freeze pane on the top row.
    worksheet.freeze_panes(5, 0)

    
    worksheet.set_column('A:A', 20, font_fmt)
    worksheet.set_column('B:B', 18, font_fmt)
    worksheet.set_column('C:C', 27, font_fmt)
    worksheet.set_column('D:F', 15, font_fmt)
    worksheet.set_column('G:G', 15, font_fmt)
    worksheet.set_column('H:H', 45, font_fmt)
    worksheet.set_column('I:K', 12, font_fmt)
    worksheet.set_column('L:L', 6.5, font_fmt)
    worksheet.set_column('M:N', 12, font_fmt)
    worksheet.set_column('O:O', 6.5, font_fmt)
    for col_num, value in enumerate(open_charge_DONS_pending_report.columns.values):
        worksheet.write(4, col_num, value, header_fmt)

# Merge 3 cells.
    worksheet.merge_range('A1:G1', deparment_name, merge_format)
    worksheet.merge_range('A2:G2', 'Pending Charges Report', merge_format)
    worksheet.merge_range('A3:G3', date1.strftime("%B %d, %Y"), merge_format)
    worksheet.merge_range('A4:G4', "", merge_format)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

print('Report Created: ' + filename)

# Create a Pandas Excel writer using XlsxWriter as the engine.
import pandas.io.formats.excel
pandas.io.formats.excel.ExcelFormatter.header_style = None


date_str = date1.strftime("%m-%d-%Y")
extension = '.xlsx'
filename = 'Pending Charges Reports as of '+ date_str +extension
writer = pd.ExcelWriter(filename, engine='xlsxwriter', datetime_format='mm/dd/yyyy',
                            date_format='mm/dd/yyyy')

# Write each dataframe to a different worksheet.
open_charge_ext_pending_report.to_excel(writer, sheet_name= date_str, index = False, startrow = 1)


workbook  = writer.book


# Add a format to use wrap the cell text.
wrap = workbook.add_format({'text_wrap': True})


#text formats
merge_format = workbook.add_format({
    'bold': True,
    'font_name': 'Times New Roman',
    'font_size': 14,
    'align': 'center',
    'valign': 'vcenter',
    'text_wrap': True,})


font_fmt = workbook.add_format({'font_name': 'Calibri', 'font_size': 10, 'text_wrap': True})

header_fmt = workbook.add_format({'font_name': 'Times New Roman',
                                  'font_size': 10,
                                  'fg_color': '#BFBFBF',
                                  'text_wrap': True,
                                 'bold' : True,
                                 'align': 'center'})

#create worksheets
worksheet = writer.sheets[date_str]


# Freeze pane on the top row.
worksheet.freeze_panes(2, 0)


worksheet.set_column('A:A', 25, font_fmt)
worksheet.set_column('B:B', 15, font_fmt)
worksheet.set_column('C:C', 15, font_fmt)
worksheet.set_column('D:D', 15, font_fmt)
worksheet.set_column('E:G', 15, font_fmt)
worksheet.set_column('H:I', 15, font_fmt)
worksheet.set_column('J:J', 12, font_fmt)
worksheet.set_column('K:K', 25, font_fmt)
worksheet.set_column('L:L', 15, font_fmt)
worksheet.set_column('M:M', 13, font_fmt)
worksheet.set_column('N:N', 15, font_fmt)
worksheet.set_column('O:O', 15, font_fmt)
worksheet.set_column('P:P', 35, font_fmt)
worksheet.set_row(0, 57, merge_format)
worksheet.set_row(1, 45, header_fmt)


text = 'Health and Human Services Commission - Civil Rights Office \
        \nPending External Employment Discrimination Charges Report \n' + date_str

# Merge 3 cells.
worksheet.merge_range('A1:P1', text, merge_format)

    # Close the Pandas Excel writer and output the Excel file.
writer.save()

print('Report Created: ' + filename)

Report created:All Non-Pending Cases as of 05-04-2021.xlsx
Report created:Weekly Pending Complaints as of 05-04-2021.xlsx
Report created:All Staff Weekly Pending Cases as of 05-04-2021.xlsx
Report created:All Staff Pending Program Mgr as of 05-04-2021.xlsx
Report created:All Staff Pending Employment Cases as of 05-04-2021.xlsx
Report created:All Past Due PSP Pending Cases as of 05-04-2021.xlsx
Report Created: Pending Cases Over 60 Days as of 05-04-2021 without 26s & 31s.xlsx
Report Created: Pending Cases Over 50 Days as of 05-04-2021 without 12-23-26-32-36.xlsx
Report Created: All FNS Pending Cases as of 05-04-2021.xlsx
Report Created: FNS Pending Cases Reg 00-07-09 as of 05-04-2021.xlsx
Report Created: FNS Pending Cases Reg 01-02-03 as of 05-04-2021.xlsx
Report Created: FNS Pending Cases Reg 04-05-06 as of 05-04-2021.xlsx
Report Created: FNS Pending Cases Reg 08-10-11 as of 05-04-2021.xlsx
Report Created: FNS Pending Cases Reg CO Staff as of 05-04-2021.xlsx
Report Created: Pending Cas

In [338]:
all_staff_weekly_pending_cases_report.columns

Index(['STAFF', 'CASE NUM', 'CASE TYPE', 'INTAKE DT', 'CASE STATUS',
       'Work Days Between', 'EXTNL AGENCY', 'DT RESPONSE', 'C-AGENCY',
       'R-AGENCY', 'Region'],
      dtype='object')

In [343]:
#create HTML pages to convert to PDF in Adobe

from IPython.display import HTML
from jinja2 import Environment, FileSystemLoader

for staff in s_list:
    report = all_staff_weekly_pending_cases_report[(all_staff_weekly_pending_cases_report['STAFF'] == staff)]
    report['EXTNL AGENCY'].fillna('-', inplace = True)
    report['DT RESPONSE'].fillna('-', inplace = True)
    region_name = str(report['Region'].drop_duplicates().to_list())[2:-2]
    report = report.drop(['Region', 'STAFF'],axis=1)
    report.sort_values('CASE NUM', inplace = True)
    report.reset_index(drop = True, inplace = True)
    report.index += 1 
    env = Environment(loader=FileSystemLoader('.'))
    template = env.get_template("mytemplate.html")
    
    env = Environment(loader=FileSystemLoader('.'))
    template = env.get_template("mytemplate.html")
    
    template_vars = {"title" : "Pending Cases "+ "for "+ staff +" - Week of " + date1.strftime("%m-%d-%Y"),
                     "summary": "Summary for " + staff,
                 "national_pivot_table": report.to_html(),
                    "count": report.shape[0],
                     "region": " Region: " + region_name,
                    "avg_workdays": int(report['Work Days Between'].mean()),
                    "oldest" : int(report['Work Days Between'].max())}
    html_out = template.render(template_vars)
    
    # to save the results
    with open("All Staff Pending Admin Complaints " + staff + ".html", "w") as fh:
        fh.write(html_out)

C:\Users\JFaulk01\Miniconda3\lib\site-packages\pandas\core\series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\Users\JFaulk01\Miniconda3\lib\site-packages\pandas\core\series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\Users\JFaulk01\Miniconda3\lib\site-packages\pandas\core\series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\User

C:\Users\JFaulk01\Miniconda3\lib\site-packages\pandas\core\series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\Users\JFaulk01\Miniconda3\lib\site-packages\pandas\core\series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\Users\JFaulk01\Miniconda3\lib\site-packages\pandas\core\series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\User

C:\Users\JFaulk01\Miniconda3\lib\site-packages\pandas\core\series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\Users\JFaulk01\Miniconda3\lib\site-packages\pandas\core\series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\Users\JFaulk01\Miniconda3\lib\site-packages\pandas\core\series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\User

C:\Users\JFaulk01\Miniconda3\lib\site-packages\pandas\core\series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\Users\JFaulk01\Miniconda3\lib\site-packages\pandas\core\series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\Users\JFaulk01\Miniconda3\lib\site-packages\pandas\core\series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\User

C:\Users\JFaulk01\Miniconda3\lib\site-packages\pandas\core\series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\Users\JFaulk01\Miniconda3\lib\site-packages\pandas\core\series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\Users\JFaulk01\Miniconda3\lib\site-packages\pandas\core\series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\User

C:\Users\JFaulk01\Miniconda3\lib\site-packages\pandas\core\series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\Users\JFaulk01\Miniconda3\lib\site-packages\pandas\core\series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\Users\JFaulk01\Miniconda3\lib\site-packages\pandas\core\series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\User